# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
!pip install simpleaudio

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image



In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message_translator = "You are an english to hebrew professional translator. return the messages as Hebrew"

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown") # the json with tickes prices

In [ ]:
# And this is included in a list of tools:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}



In [ ]:
reservation_function = {
    "name": "store_reservation",
    "description": "Store a reservation for a return ticket, including city, price, and user name, in a text file.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city the customer wants to reserve a ticket for",
            },
            "price": {
                "type": "string",
                "description": "The price of the return ticket",
            },
            "first_name": {
                "type": "string",
                "description": "The name of the customer making the reservation",
            },
        },
        # The LLM is smart enough to extract the user's name from conversation context 
        # (e.g. "My name is Alex") and automatically insert it into the 'first_name' parameter
        "required": ["destination_city", "price", "first_name"], 
        "additionalProperties": False
    }
}


In [ ]:
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": reservation_function}
]

In [ ]:
# FIXED: Updated function signature to include first_name
def store_reservation(destination_city, price, first_name):
    print(f"Tool store_reservation called for {destination_city} with price {price} for {first_name}")
    print(f"Drawing the ticket for {first_name}. please wait...")
    with open("reservations.txt", "a") as f:
        f.write(f"{destination_city}: {price} - {first_name}\n")
    return "Reservation confirmed"


In [ ]:
# FIXED: Simplified handle_tool_call function
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    tool_name = tool_call.function.name

    if tool_name == "get_ticket_price":
        city = arguments.get('destination_city')
        result = get_ticket_price(city)
        response_data = {"destination_city": city, "price": result}
        city_for_image = None

    elif tool_name == "store_reservation":
        city = arguments.get('destination_city')
        price = arguments.get('price')
        first_name = arguments.get('first_name')
        
        # The LLM should extract the name from conversation context
        # If it doesn't, the function will fail and the LLM will ask for it
        result = store_reservation(city, price, first_name)
        response_data = {"destination_city": city, "price": price, "first_name": first_name, "result": result}
        city_for_image = city

    else:
        response_data = {"error": "Unknown tool"}
        city_for_image = None

    response = {
        "role": "tool",
        "content": json.dumps(response_data),
        "tool_call_id": tool_call.id
    }

    return response, city_for_image

In [ ]:
# FIXED: Artist function should return the image, not display it
def artist(city, first_name):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt = f"""Design an airline boarding pass ticket. THE MOST IMPORTANT REQUIREMENT: 
The text "PASSENGER NAME: {first_name.upper()}" must be prominently displayed at the top in large, readable letters.
Below this, show flight details to {city}, and at the bottom include artwork of {city} landmarks.
This is critical: the name {first_name.upper()} MUST be visible and readable on the ticket.""",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    image = Image.open(BytesIO(image_data))
    resized_image = image.resize((896, 512))  # Example size
    return resized_image


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]   
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    
    # If tool_calls, handle tool and re-call model
    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        tool_response, city = handle_tool_call(assistant_message)

        messages.append(assistant_message)
        messages.append(tool_response)

        if city:
            tool_data = json.loads(tool_response["content"])
            first_name = tool_data.get("first_name")
            if first_name:
                image = artist(city, first_name)
                from IPython.display import display
                display(image)

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    # ✅ Get the final assistant reply (both for tools and regular messages)
    content = response.choices[0].message.content

    # send content to talker
    talker(response.choices[0].message.content)


    # ✅ Translate *the assistant reply*, not the user message
    translated_prompt = [
        {"role": "system", "content": system_message_translator},
        {"role": "user", "content": content}
    ]
    translated_response = openai.chat.completions.create(model=MODEL, messages=translated_prompt)
    translated_content = translated_response.choices[0].message.content

    return content, translated_content


In [ ]:
def gradio_create_chat_window():
    with gr.Blocks() as ui:
        with gr.Row():
            with gr.Column(scale=2):
                chatbot = gr.Chatbot(label="FlightAI Assistant", height=500, type="messages")
                input_box = gr.Textbox(label="Type your message", placeholder="How much is a ticket to Tokyo?")
                send_button = gr.Button("Send")
            with gr.Column(scale=1):
                translated_output = gr.Textbox(label="Translation", lines=20, rtl=True)
    
        def handle_chat(user_message, history, translation_history):
            history = history or []
            translation_history = translation_history or ""
        
            english_reply, translated_reply = chat(user_message, history)
        
            # Update chat history
            history.append({"role": "user", "content": user_message})
            history.append({"role": "assistant", "content": english_reply})
        
            # Append new translation to the existing translation history
            translation_history += f"\n👤 {user_message}\n🤖 {translated_reply}\n"
        
            return "", history, translation_history


    
        def translate(text):
            # You can replace this with actual translation (Google Translate API, etc.)
            return f"[HE] {text}"  # fake example
    
        send_button.click(
            handle_chat,
            inputs=[input_box, chatbot, translated_output],
            outputs=[input_box, chatbot, translated_output]
        )

        input_box.submit(
            handle_chat,
            inputs=[input_box, chatbot, translated_output],
            outputs=[input_box, chatbot, translated_output]
        )
    
    ui.launch()



In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

# talker("Well, hi there")

In [ ]:
gradio_create_chat_window()